In [2]:
import sys
sys.path.append('..') 
from helper_fun import *
from sklearn.cluster import AgglomerativeClustering

def tanimoto_distance_matrix(fp_list):
    """Calculate distance matrix for fingerprint list"""
    dissimilarity_matrix = []
    # Notice how we are deliberately skipping the first and last items in the list
    # because we don't need to compare them against themselves
    for i in range(1, len(fp_list)):
        # Compare the current fingerprint against all the previous ones in the list
        similarities = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        # Since we need a distance matrix, calculate 1-x for every element in similarity matrix
        dissimilarity_matrix.extend([1 - x for x in similarities])
    return dissimilarity_matrix

#define a function that takes in a list of fingerprints and a cutoff value and returns a list of clusters using agglomerative clustering
def hierarchical_cluster_fingerprints(fingerprints, cutoff=0.2):
    """Cluster fingerprints
    Parameters:
        fingerprints
        cutoff: threshold for the clustering, 0.2 is usual
    """
    # Calculate the Tanimoto distance matrix
    distance_matrix = tanimoto_distance_matrix(fingerprints)
    # Perform agglomerative clustering
    cluster = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=cutoff)
    # Return the cluster labels
    return cluster.fit_predict(distance_matrix)

# Note: unable to get hierarchical clustering to work
### trouble with dimension of array we are expecting vs feeding for the cluster.fit_predict
### Please look at the hierarchical_cluster_fingerprints function

In [16]:
# # Create molecules from SMILES and store in array
# compounds = pd.read_csv("../COVID_MOONSHOT/compounds_filtered.csv")
# compounds_list = []

# for _, chembl_id, smiles in compounds[["CID", "SMILES"]].itertuples():
#     compounds_list.append((Chem.MolFromSmiles(smiles), chembl_id))
# rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)


# fingerprints = fingerprints = [rdkit_gen.GetFingerprint(mol) for mol, idx in compounds_list]
# hierarchical_cluster_fingerprints(fingerprints, cutoff=0.2)